In [ ]:
# -*- coding: utf-8 -*-
"""
Код ячейки Google Colab для установки зависимостей,
инициализации компонентов и запуска интерактивного интерфейса
фреймворка Binance US Data Framework.

Используется минимальный набор установок, полагаясь на предустановленные
версии numpy, pandas, matplotlib в Colab.
"""

# -------------------------------------------
# ШАГ 1: Установка только необходимых зависимостей
# -------------------------------------------
print("Установка минимально необходимых зависимостей...")
# Устанавливаем только те пакеты, которых нет в Colab или для которых важна версия
!pip install python-binance ipywidgets duckdb --quiet
!pip install pandas matplotlib numpy==1.24.3 --quiet
print("Зависимости установлены.")
# -------------------------------------------
# ШАГ 2: Установка вашего фреймворка из Git
# -------------------------------------------
# Используем URL репозитория
GIT_REPO_URL = 'git+https://github.com/St0rmMaster/binance_data_framework.git'

print(f"Установка фреймворка из {GIT_REPO_URL}...")
# Pip попытается установить зависимости фреймворка (numpy, pandas, matplotlib),
# но, скорее всего, увидит, что они уже установлены, и использует существующие версии Colab.
!pip install --no-cache-dir {GIT_REPO_URL} --quiet
print("Фреймворк установлен из Git.")

In [ ]:
# -------------------------------------------
# ШАГ 3: Импорт и запуск (БЕЗ перезапуска среды выполнения)
# -------------------------------------------
# Попробуем запустить без перезапуска. Если ошибка NumPy вернется,
# значит, фреймворк или python-binance 1.0.19 несовместимы с NumPy из Colab,
# и потребуется явная установка совместимой версии NumPy + перезапуск.

print("Импорт библиотек и запуск интерфейса...")

# Импортируем необходимые классы
try:
    from binance_data_framework import BinanceUSClient, LocalDataManager, DataDownloaderUI
    print("Фреймворк binance_data_framework успешно импортирован.")
except ImportError as e:
    # Эта ошибка может возникнуть, если установка фреймворка не удалась
    print(f"Ошибка импорта фреймворка: {e}")
    raise
except ValueError as e:
    # Ловим ошибку NumPy, если она все еще возникает
    if 'numpy.dtype size changed' in str(e):
        print("\n❌ Ошибка совместимости NumPy все еще присутствует!")
        print("Похоже, фреймворк или его зависимости (возможно, python-binance 1.0.19)")
        print("несовместимы с версией NumPy, предустановленной в Colab.")
        print("В этом случае может потребоваться установка совместимой версии NumPy")
        print("и перезапуск среды выполнения (см. предыдущий вариант решения).")
    else:
        print(f"Произошла неожиданная ошибка ValueError: {e}")
    raise
except Exception as e:
    print(f"Произошла неожиданная ошибка при импорте: {e}")
    raise


# Другие необходимые импорты
import os
from datetime import datetime
from IPython.display import display, clear_output

# -------------------------------------------
# ШАГ 4: Настройка и инициализация компонентов
# -------------------------------------------

# --- 4.1: Клиент Binance API ---
# from google.colab import userdata
# api_key = userdata.get('YOUR_BINANCE_US_API_KEY')
# api_secret = userdata.get('YOUR_BINANCE_US_API_SECRET')
api_key = None
api_secret = None

print("Инициализация клиента Binance US API...")
api_client = BinanceUSClient(api_key=api_key, api_secret=api_secret)
connection_status = api_client.connect()
if connection_status:
    print("✅ Успешное подключение к Binance US API.")
else:
    print("⚠️ Ошибка подключения к Binance US API. Функциональность будет ограничена.")

# --- 4.2: Менеджер локальной базы данных ---
db_filename = 'binance_ohlcv_data.db'
print(f"Инициализация менеджера локальной базы данных (SQLite: {db_filename})...")
db_manager = LocalDataManager(db_path=db_filename)

# -------------------------------------------
# ШАГ 5: Запуск интерактивного интерфейса
# -------------------------------------------
print("Создание и отображение интерактивного интерфейса...")
clear_output(wait=True) # Очищаем вывод перед показом UI
ui = DataDownloaderUI(api_client=api_client, db_manager=db_manager)
ui.display()
print("\nИнтерфейс готов к использованию.")
print(f"Локальные данные будут храниться в файле: {os.path.abspath(db_filename)}")